## 필요 라이브러리 불러오기

In [3]:
import pandas as pd
import numpy as np
import pymysql

## DB 저장: yorehalre DB

### 01.nutrition_limit tbl  
    - 성별&연령대별 영양성분 제한기준표

In [73]:
excel_file ='../src/영양성분 섭취기준.xlsx'
df = pd.read_excel(excel_file)
df.head()

,성별,연령,에너지(㎉),탄수화물(g),총 식이섬유(g),총당류(g),단백질(g),지방(g),콜레스테롤(㎎),총 포화 지방산(g),...,칼륨(㎎),철(㎎),마그네슘(㎎),비타민 A(μg RAE),비타민 D(㎍),비타민 E(㎎ α-TE),비타민 K(㎍),비타민 C(㎎),엽산(㎍),비타민 B12(㎍)
0,남자,19-29,2600.0,130.0,30.0,65.0,65.0,NaN,300.0,23.111111,...,3500.0,8-45,360-700,570-800-3000,10-100,12-540,75.0,75-100-2000,400-1000,2.0
1,남자,30-49,2500.0,130.0,30.0,62.5,65.0,NaN,300.0,22.222222,...,3500.0,8-45,370-720,560-800-3000,10-100,12-540,75.0,75-100-2000,400-1000,2.0
2,남자,50-64,2200.0,130.0,30.0,55.0,60.0,NaN,300.0,19.555556,...,3500.0,8-45,370-720,530-750-3000,10-100,12-540,75.0,75-100-2000,400-1000,2.0
3,남자,65-74,2000.0,130.0,25.0,50.0,60.0,NaN,300.0,17.777778,...,3500.0,7-45,370-720,510-700-3000,15-100,12-540,75.0,75-100-2000,400-1000,2.0
4,남자,75 이상,1900.0,130.0,25.0,47.5,60.0,NaN,300.0,16.888889,...,3500.0,7-45,370-720,500-700-3000,15-100,12-540,75.0,75-100-2000,400-1000,2.0


In [74]:
df = df.iloc[:10] # 필요행만 추출
df.columns = df.columns.str.replace(' ','') # 열이름에 공백 제거 (sql열이름에 공백 불허해서)

df

,성별,연령,에너지(㎉),탄수화물(g),총식이섬유(g),총당류(g),단백질(g),지방(g),콜레스테롤(㎎),총포화지방산(g),...,칼륨(㎎),철(㎎),마그네슘(㎎),비타민A(μgRAE),비타민D(㎍),비타민E(㎎α-TE),비타민K(㎍),비타민C(㎎),엽산(㎍),비타민B12(㎍)
0,남자,19-29,2600.0,130.0,30.0,65.0,65.0,NaN,300.0,23.111111,...,3500.0,8-45,360-700,570-800-3000,10-100,12-540,75.0,75-100-2000,400-1000,2.0
1,남자,30-49,2500.0,130.0,30.0,62.5,65.0,NaN,300.0,22.222222,...,3500.0,8-45,370-720,560-800-3000,10-100,12-540,75.0,75-100-2000,400-1000,2.0
2,남자,50-64,2200.0,130.0,30.0,55.0,60.0,NaN,300.0,19.555556,...,3500.0,8-45,370-720,530-750-3000,10-100,12-540,75.0,75-100-2000,400-1000,2.0
3,남자,65-74,2000.0,130.0,25.0,50.0,60.0,NaN,300.0,17.777778,...,3500.0,7-45,370-720,510-700-3000,15-100,12-540,75.0,75-100-2000,400-1000,2.0
4,남자,75 이상,1900.0,130.0,25.0,47.5,60.0,NaN,300.0,16.888889,...,3500.0,7-45,370-720,500-700-3000,15-100,12-540,75.0,75-100-2000,400-1000,2.0
5,여자,19-29,2000.0,130.0,20.0,50.0,55.0,NaN,300.0,17.777778,...,3500.0,11-45,280-630,460-650-3000,10-100,12-540,65.0,75-100-2000,400-1000,2.0
6,여자,30-49,1900.0,130.0,20.0,47.5,50.0,NaN,300.0,16.888889,...,3500.0,11-45,280-630,450-650-3000,10-100,12-540,65.0,75-100-2000,400-1000,2.0
7,여자,50-64,1700.0,130.0,20.0,42.5,50.0,NaN,300.0,15.111111,...,3500.0,6-45,280-630,430-600-3000,10-100,12-540,65.0,75-100-2000,400-1000,2.0
8,여자,65-74,1600.0,130.0,20.0,40.0,50.0,NaN,300.0,14.222222,...,3500.0,6-45,280-630,410-600-3000,15-100,12-540,65.0,75-100-2000,400-1000,2.0
9,여자,75 이상,1500.0,130.0,20.0,37.5,50.0,NaN,300.0,13.333333,...,3500.0,5-45,280-630,410-600-3000,15-100,12-540,65.0,75-100-2000,400-1000,2.0


In [75]:
# (성별,연령)기준으로 데이터프레임 재구조화
melt_df = pd.melt(df, id_vars=['성별','연령'])
melt_df = melt_df.rename(columns={'성별':'sex', '연령':'age_range','variable':'nutrition'}) # 열이름 변경

melt_df

,sex,age_range,nutrition,value
0,남자,19-29,에너지(㎉),2600.0
1,남자,30-49,에너지(㎉),2500.0
2,남자,50-64,에너지(㎉),2200.0
3,남자,65-74,에너지(㎉),2000.0
4,남자,75 이상,에너지(㎉),1900.0
...,...,...,...,...
215,여자,19-29,비타민B12(㎍),2.0
216,여자,30-49,비타민B12(㎍),2.0
217,여자,50-64,비타민B12(㎍),2.0
218,여자,65-74,비타민B12(㎍),2.0


In [76]:
# 영양소 종류 확인
print(melt_df.nutrition.unique(), f'총 영양소 개수 : {len(melt_df.nutrition.unique())}개', sep='\n\n')

['에너지(㎉)' '탄수화물(g)' '총식이섬유(g)' '총당류(g)' '단백질(g)' '지방(g)' '콜레스테롤(㎎)'
 '총포화지방산(g)' '트랜스지방산(g)' 'EPA+DHA(g)' '칼슘(㎎)' '나트륨(㎎)' '칼륨(㎎)' '철(㎎)'
 '마그네슘(㎎)' '비타민A(μgRAE)' '비타민D(㎍)' '비타민E(㎎α-TE)' '비타민K(㎍)' '비타민C(㎎)'
 '엽산(㎍)' '비타민B12(㎍)']

총 영양소 개수 : 22개


In [77]:
# value값에서 기준치 분리 후, 새로운 열의 값으로 넣기
    # ex) 철의 value는 8-45인데, 8은 결핍기준(lack_nutrition)이 되고, 45는 과다기준(over_nutrition)값이 된다.

lack_nutri = ['탄수화물(g)','단백질(g)', '지방(g)','총식이섬유(g)','비타민K(㎍)','비타민B12(㎍)','칼륨(㎎)','EPA+DHA(g)']
over_nutri= ['에너지(㎉)','콜레스테롤(㎎)','총포화지방산(g)','트랜스지방산(g)','총당류(g)','나트륨(㎎)']
lack_over_nutri = ['비타민D(㎍)','비타민E(㎎α-TE)','엽산(㎍)','철(㎎)', '마그네슘(㎎)']
danger_nutri= ['비타민A(μgRAE)','비타민 C(㎎)','칼슘(㎎)']

for index, row in melt_df.iterrows():
    if row['nutrition'] in lack_nutri:
        melt_df.loc[index,'lack_standard'] = row['value']
    elif row['nutrition'] in over_nutri:
        melt_df.loc[index,'over_standard'] = row['value']
    elif row['nutrition'] in lack_over_nutri:
        melt_df.loc[index,'lack_standard'] = row['value'].split('-')[0]
        melt_df.loc[index,'over_standard'] = row['value'].split('-')[1]
    elif row['nutrition'] in danger_nutri:
        melt_df.loc[index,'lack_standard'] = row['value'].split('-')[0]
        melt_df.loc[index,'over_standard'] = row['value'].split('-')[1]
        melt_df.loc[index,'danger_standard'] = row['value'].split('-')[2]
melt_df

C:\Users\dldkr\AppData\Local\Temp\ipykernel_27580\2902918236.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '650' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  melt_df.loc[index,'lack_standard'] = row['value'].split('-')[0]
C:\Users\dldkr\AppData\Local\Temp\ipykernel_27580\2902918236.py:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '800' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  melt_df.loc[index,'over_standard'] = row['value'].split('-')[1]


,sex,age_range,nutrition,value,over_standard,lack_standard,danger_standard
0,남자,19-29,에너지(㎉),2600.0,2600.0,NaN,NaN
1,남자,30-49,에너지(㎉),2500.0,2500.0,NaN,NaN
2,남자,50-64,에너지(㎉),2200.0,2200.0,NaN,NaN
3,남자,65-74,에너지(㎉),2000.0,2000.0,NaN,NaN
4,남자,75 이상,에너지(㎉),1900.0,1900.0,NaN,NaN
...,...,...,...,...,...,...,...
215,여자,19-29,비타민B12(㎍),2.0,NaN,2.0,NaN
216,여자,30-49,비타민B12(㎍),2.0,NaN,2.0,NaN
217,여자,50-64,비타민B12(㎍),2.0,NaN,2.0,NaN
218,여자,65-74,비타민B12(㎍),2.0,NaN,2.0,NaN


In [78]:
# 불필요 열 제거 & 열 재배열
final_df = melt_df.copy()
final_df = final_df[['sex','age','nutrition','lack_standard', 'over_standard','danger_standard']]
final_df['age_range'] = final_df['age_range'].str.replace(' 이상','-')

# 숫자형 데이터만 정수형으로 반올림 처리 (시각화할때, 기준값으로 쓰이므로 소수점은 시각화했을 때 깔끔하지 못해서)
final_df_numeric = final_df.select_dtypes(include=np.number)
final_df_rounded = final_df_numeric.round().astype('Int64')
final_df[final_df_rounded.columns] = final_df_rounded

final_df

,sex,age_range,nutrition,over_standard,lack_standard,danger_standard
0,남자,19-29,에너지(㎉),2600.0,NaN,NaN
1,남자,30-49,에너지(㎉),2500.0,NaN,NaN
2,남자,50-64,에너지(㎉),2200.0,NaN,NaN
3,남자,65-74,에너지(㎉),2000.0,NaN,NaN
4,남자,75-,에너지(㎉),1900.0,NaN,NaN
...,...,...,...,...,...,...
215,여자,19-29,비타민B12(㎍),NaN,2.0,NaN
216,여자,30-49,비타민B12(㎍),NaN,2.0,NaN
217,여자,50-64,비타민B12(㎍),NaN,2.0,NaN
218,여자,65-74,비타민B12(㎍),NaN,2.0,NaN


In [61]:
## DB연결후, 저장

import pymysql

conn = pymysql.connect(host='localhost', user='root', password='1111',\
    db = 'yorehalre', charset='utf8')
cur= conn.cursor()

cur.execute(""" 
            create table if not exists nutrition_limit ( 
            sex CHAR(2) NOT NULL,
            age_range VARCHAR(10) NOT NULL,
            nutrition VARCHAR(20) NOT NULL,
            lack_standard INT,
            over_standard INT,
            danger_standard INT,
            PRIMARY KEY (sex,age_range,nutrition)
            )
            """)

for idx,row in final_df.iterrows():
    try:
        sql = "INSERT INTO nutrition_limit(sex, age_range, nutrition, lack_standard, over_standard, danger_standard) values(%s, %s, %s, %s, %s, %s)"
        
        val = (row['sex'] if not pd.isnull(row['sex']) else None, # NaN값 None 값으로 처리 (mysql에 NaN값 불허)
               row['age_range'] if not pd.isnull(row['age_range']) else None,
               row['nutrition'] if not pd.isnull(row['nutrition']) else None,
               row['lack_standard'] if not pd.isnull(row['lack_standard']) else None,
               row['over_standard'] if not pd.isnull(row['over_standard']) else None,
               row['danger_standard'] if not pd.isnull(row['danger_standard']) else None)
        
        cur.execute(sql, val)
        
    except Exception as e:
        print("Error:", e)
        continue

conn.commit()
conn.close()

### 02.food_nutrition tbl
    - 모든 음식 영양성분표

In [80]:
excel_file ='../src/food_nutrient.xlsx'
df = pd.read_excel(excel_file)
df.head()

,식품명,1회제공량,에너지(㎉),에너지(kj),수분(g),수분(%),단백질(g),지방(g),탄수화물(g),총당류(g),...,EPA와 DHA의 합(㎎),오메가 3 지방산(g),오메가 6 지방산(g),트랜스 지방산(g),철(㎎),EPA+DHA(g),비타민 A(μg RAE),비타민 B3(㎎),메티오닌+시스테인(㎎),페닐알라닌+티로신(㎎)
0,닭갈비,400,595.610,0,276.400,0.0,45.900,25.800,44.900,21.200,...,0,0.0,0.0,0.200,0.003380,0.016,394.171667,1.230,1185.770,3140.654
1,쌀국수,600,320.320,0,520.400,0.0,15.600,4.200,55.100,4.300,...,0,0.0,0.0,0.100,0.001510,0.000,17.420000,1.810,476.668,1098.648
2,돼지고기볶음,200,350.673,0,134.172,0.0,22.961,20.665,18.210,8.201,...,0,0.0,0.0,0.151,0.015974,0.000,217.256000,4.841,0.000,0.000
3,김치찌개,400,243.032,0,351.450,0.0,15.069,15.005,11.929,3.025,...,0,0.0,0.0,0.030,0.015184,0.000,186.184333,6.595,0.000,0.000
4,양념치킨,200,552.000,0,92.100,0.0,35.500,26.800,42.300,12.500,...,0,0.0,0.0,0.200,0.001470,0.000,63.310000,8.620,1316.390,2647.650


In [81]:
# 열 전처리

# 사용할 영양성분 열만 추출
df = df[['식품명', '1회제공량', '에너지(㎉)','탄수화물(g)','총 식이섬유(g)','총당류(g)','단백질(g)','지방(g)','콜레스테롤(㎎)','총 포화 지방산(g)','트랜스 지방산(g)',\
    'EPA+DHA(g)','칼슘(㎎)','나트륨(㎎)','칼륨(㎎)','철(㎎)','마그네슘(㎎)','비타민 A(μg RAE)','비타민 D(㎍)','비타민 E(㎎ α-TE)','비타민 K(㎍)','비타민 C(㎎)','엽산(㎍)',
    '비타민 B12(㎍)']]
df.rename(columns={'식품명':'food_name'},inplace=True) # 열 이름 변경
df.columns = df.columns.str.replace(' ','') # 열 이름 공백제거

df.head()

,food_name,1회제공량,에너지(㎉),탄수화물(g),총식이섬유(g),총당류(g),단백질(g),지방(g),콜레스테롤(㎎),총포화지방산(g),...,칼륨(㎎),철(㎎),마그네슘(㎎),비타민A(μgRAE),비타민D(㎍),비타민E(㎎α-TE),비타민K(㎍),비타민C(㎎),엽산(㎍),비타민B12(㎍)
0,닭갈비,400,595.610,44.900,11.600,21.200,45.900,25.800,193.400,6.000,...,1200.240,0.003380,104.42,394.171667,0.0,6.82,0.0,5.540,108.13,1.12
1,쌀국수,600,320.320,55.100,1.600,4.300,15.600,4.200,33.600,1.400,...,253.260,0.001510,36.11,17.420000,0.0,0.90,0.0,0.370,42.22,1.34
2,돼지고기볶음,200,350.673,18.210,3.020,8.201,22.961,20.665,57.870,5.119,...,568.054,0.015974,0.00,217.256000,0.0,0.00,0.0,0.746,0.00,0.00
3,김치찌개,400,243.032,11.929,8.417,3.025,15.069,15.005,25.178,2.756,...,639.691,0.015184,0.00,186.184333,0.0,0.00,0.0,0.000,0.00,0.00
4,양념치킨,200,552.000,42.300,2.100,12.500,35.500,26.800,137.790,5.400,...,495.730,0.001470,51.32,63.310000,0.0,8.18,0.0,0.200,33.24,0.31


In [66]:
# 결측값 확인
print(df.isna().sum())

food_name      0
1회제공량          0
에너지(㎉)         0
탄수화물(g)        0
총식이섬유(g)       0
총당류(g)         0
단백질(g)         0
지방(g)          0
콜레스테롤(㎎)       0
총포화지방산(g)      0
트랜스지방산(g)      0
EPA+DHA(g)     0
칼슘(㎎)          0
나트륨(㎎)         0
칼륨(㎎)          0
철(㎎)           0
마그네슘(㎎)        0
비타민A(μgRAE)    0
비타민D(㎍)        0
비타민E(㎎α-TE)    0
비타민K(㎍)        0
비타민C(㎎)        0
엽산(㎍)          0
비타민B12(㎍)      0
dtype: int64


In [69]:
## DB연결후, 저장

import pymysql

conn = pymysql.connect(host='localhost', user='root', password='1111',\
    db = 'yorehalre', charset='utf8')
cur= conn.cursor()

# mysql의 열에 특수문자()괄호를 넣기 위해, 백틱(`) 처리
cur.execute(""" 
            create table if not exists food_nutrition ( 
            food_name VARCHAR(100) NOT NULL,
            1회제공량 INT NOT NULL,
            `에너지(㎉)` FLOAT NOT NULL,
            `탄수화물(g)` FLOAT NOT NULL,
            `총식이섬유(g)` FLOAT NOT NULL,
            `총당류(g)` FLOAT NOT NULL,
            `단백질(g)` FLOAT NOT NULL,
            `지방(g)` FLOAT NOT NULL,
            `콜레스테롤(㎎)` FLOAT NOT NULL,
            `총포화지방산(g)` FLOAT NOT NULL,
            `트랜스지방산(g)` FLOAT NOT NULL,
            `EPA+DHA(g)` FLOAT,
            `칼슘(㎎)` FLOAT,
            `나트륨(㎎)` FLOAT NOT NULL,
            `칼륨(㎎)` FLOAT,
            `철(㎎)` FLOAT,
            `마그네슘(㎎)` FLOAT,
            `비타민A(μgRAE)` FLOAT,
            `비타민D(㎍)` FLOAT,
            `비타민E(㎎α-TE)` FLOAT,
            `비타민K(㎍)` FLOAT,
            `비타민C(㎎)` FLOAT,
            `엽산(㎍)` FLOAT,
            `비타민B12(㎍)` FLOAT,
            PRIMARY KEY(food_name, 1회제공량)
            )
            """ 
            )
            
# DataFrame의 iterate 대신, executemany를 사용하여 한 번에 여러행을 삽입.
data = df.values.tolist() # 데이터프레임 > 리스트 변환
placeholders = ', '.join(['%s'] * len(df.columns))
          
try: 
    cur.executemany(f"INSERT INTO food_nutrition VALUES ({placeholders})", data)        
except Exception as e:
    print("Error:", e)

conn.commit()
conn.close()

### 03.recipe_info tbl
    - 모든 음식 레시피 저장소 + 음식사진 링크

In [31]:
excel_file ='../src/weaviate_data.csv'
df = pd.read_csv(excel_file)

df.head()

,recipe_name,summary,ingredient_name,full_step,category,image_link
0,우엉양배추볶음밥,찬밥이 기다려지는 우엉양배추볶음밥입니다.,[재료] 찬 밥 적당량 양배추 적당량 우엉 적당량 느타리버섯 조금 다시마 ...,찬밥을 준비해주세요. 양배추와 우엉을 준비해주시고 양배추는 깨끗히 씻어서 잘게 잘라...,저칼로리 다이어트,https://recipe1.ezmember.co.kr/cache/recipe/20...
1,아삭아삭 연근장아찌 만들기,연근은 비타민C와 식이섬유소가 풍부하고 빈혈예방에 탁월하고 코피가 자주나는 아이들에...,[재료] 연근 [양념비율(200ml 계량컵 기준)] 간장 1 물 1 설탕 0...,연근은 감자칼로 껍질을 벗겨주었어요. 슥슥 0.5mm정도 잘라주었답니다. 찬물에 식...,저칼로리 다이어트,https://recipe1.ezmember.co.kr/cache/recipe/20...
2,아보카도 스프레드 연어 샌드위치,근사한 오픈샌드위치 만드는 법을 알려드립니다.,[재료] 식빵 1장 훈제연어 슬라이스 2쪽 오이 1/2개 크랜베리 약간 [양...,오이는 필러로 얇게 슬라이스해 주세요. 식빵은 그릴 팬에서 앞뒤로 바삭하게 구워 주...,저칼로리 다이어트,https://recipe1.ezmember.co.kr/cache/recipe/20...
3,아보카도 멜론 냉수프 만들기,만들기 간단하고 맛이 고급스러운 멜론 냉수프,[재료] 아보카도 1/4개 멜론 1/4개 우유(농도 조절용으로) 약간,"잘 익은 멜론과 아보카도, 우유를 준비해 주세요. 우유는 농도 조절용으로 사용하셔도...",저칼로리 다이어트,https://recipe1.ezmember.co.kr/cache/recipe/20...
4,훈제연어샐러드 만드는법,연어는 어떻게 먹어도 좋지요~ 다양한 요리 방법 중 이번엔 샐러드로 만들어 보았어요...,[재료] 훈제연어 180g 양상추 치커리 양파 방울토마토 블랙올리브 크루...,연어는 진공포장 그대로 찬물에 담갔다 해동한 후 비닐을 벗겨주세요. 채소는 찬물에 ...,저칼로리 다이어트,https://recipe1.ezmember.co.kr/cache/recipe/20...


In [32]:
# ! pip install sqlalchemy

In [35]:
df.isna().sum()

recipe_name         0
summary            39
ingredient_name    32
full_step          34
category            0
image_link          0
dtype: int64

In [37]:
# DataFrame에서 NaN 값을 None으로 대체
df.replace({np.nan: None}, inplace=True)
df

,recipe_name,summary,ingredient_name,full_step,category,image_link
0,우엉양배추볶음밥,찬밥이 기다려지는 우엉양배추볶음밥입니다.,[재료] 찬 밥 적당량 양배추 적당량 우엉 적당량 느타리버섯 조금 다시마 ...,찬밥을 준비해주세요. 양배추와 우엉을 준비해주시고 양배추는 깨끗히 씻어서 잘게 잘라...,저칼로리 다이어트,https://recipe1.ezmember.co.kr/cache/recipe/20...
1,아삭아삭 연근장아찌 만들기,연근은 비타민C와 식이섬유소가 풍부하고 빈혈예방에 탁월하고 코피가 자주나는 아이들에...,[재료] 연근 [양념비율(200ml 계량컵 기준)] 간장 1 물 1 설탕 0...,연근은 감자칼로 껍질을 벗겨주었어요. 슥슥 0.5mm정도 잘라주었답니다. 찬물에 식...,저칼로리 다이어트,https://recipe1.ezmember.co.kr/cache/recipe/20...
2,아보카도 스프레드 연어 샌드위치,근사한 오픈샌드위치 만드는 법을 알려드립니다.,[재료] 식빵 1장 훈제연어 슬라이스 2쪽 오이 1/2개 크랜베리 약간 [양...,오이는 필러로 얇게 슬라이스해 주세요. 식빵은 그릴 팬에서 앞뒤로 바삭하게 구워 주...,저칼로리 다이어트,https://recipe1.ezmember.co.kr/cache/recipe/20...
3,아보카도 멜론 냉수프 만들기,만들기 간단하고 맛이 고급스러운 멜론 냉수프,[재료] 아보카도 1/4개 멜론 1/4개 우유(농도 조절용으로) 약간,"잘 익은 멜론과 아보카도, 우유를 준비해 주세요. 우유는 농도 조절용으로 사용하셔도...",저칼로리 다이어트,https://recipe1.ezmember.co.kr/cache/recipe/20...
4,훈제연어샐러드 만드는법,연어는 어떻게 먹어도 좋지요~ 다양한 요리 방법 중 이번엔 샐러드로 만들어 보았어요...,[재료] 훈제연어 180g 양상추 치커리 양파 방울토마토 블랙올리브 크루...,연어는 진공포장 그대로 찬물에 담갔다 해동한 후 비닐을 벗겨주세요. 채소는 찬물에 ...,저칼로리 다이어트,https://recipe1.ezmember.co.kr/cache/recipe/20...
...,...,...,...,...,...,...
869,절대 후회없는 깻잎김치 황금레시피^^,입맛 없을때 깻잎향이 입안가득채워져서 자꾸만 손이가는 깻잎김치예요.\n아주 간단하고...,[재료] 깻잎 30장 [양념] 간장 7T 고춧가루 1T 다진 마늘 0.5T ...,양념장을 먼저 준비해주세요.당근과 대파는 얇고 작게 채썰어주세요.양념장 재료들을 모...,치매예방,https://recipe1.ezmember.co.kr/cache/recipe/20...
870,바라깻잎으로 만든~ 깻잎나물- 딸에게# / 깻잎나물 부드럽게 하는 방법 /여름 건강식,None,[재료] 깻잎순 300g 홍고추 1/2개 다진파 1T 다진마늘 1/2T 집간...,None,치매예방,https://recipe1.ezmember.co.kr/cache/recipe/20...
871,참치깻잎밥전&참치깻잎치즈밥전,비오는날은 전이 생각나잖아요 오늘은 향긋한 깻잎과 참치와 밥을 섞어 다른반찬 필요없...,[재료] 밥 1공기 네모참치 1캔 빨강파프리카 1개 깻잎 20장 달걀 2개 ...,"네모참치,깻잎,파프리카 준비해주시고 깻잎,파프리카 잘게 썰어주세요. 참치캔은 기름기...",치매예방,https://recipe1.ezmember.co.kr/cache/recipe/20...
872,새우카레덮밥,간단하게 먹을수있는 요리가 없을까 고민하다가 냉동고를 열어보니 새우가 있어서 만들어봤어요,[재료] 칵테일 새우 15마리 양파 1/2개 밥 2공기 [양념] 카레가루 3...,재료와 양념장을 준비해주시고 양파는 채썰어주세요 후라이팬에 올리브유 1T를 넣고 마...,치매예방,https://recipe1.ezmember.co.kr/cache/recipe/20...


In [39]:
## DB연결후, 저장

import pymysql

conn = pymysql.connect(host='localhost', user='root', password='1111', db = 'yorehalre', charset='utf8')
cur= conn.cursor()

# mysql의 열에 특수문자()괄호를 넣기 위해, 백틱(`) 처리
cur.execute(""" 
            CREATE TABLE IF NOT EXISTS recipe_info (
            recipe_name VARCHAR(512) PRIMARY KEY,
            summary LONGTEXT,
            ingredient_name VARCHAR(512),
            full_step LONGTEXT,
            category VARCHAR(50),
            recipe_image_link VARCHAR(512)
            )
            """
) 
            
# DataFrame의 iterate 대신, executemany를 사용하여 한 번에 여러행을 삽입.
data = df.values.tolist() # 데이터프레임 > 리스트 변환
placeholders = ', '.join(['%s'] * len(df.columns))
          
try: 
    cur.executemany(f"INSERT INTO recipe_info VALUES ({placeholders})", data)        
except Exception as e:
    print("Error:", e)

conn.commit()
conn.close()

Error: (1062, "Duplicate entry '우렁이 죽순초무침' for key 'recipe_info.PRIMARY'")
